# Description

An example of `snort++` (https://www.snort.org/snort3) network Intrusion Detection and Prevention System (ID/IPS) deployed on an endpoint Apache host.

`snort` belongs to the class of rule-based network IDS/IPS systems. The `snort` rules work similarly to virus signatures:
like a virus scanner, `snort` can only detect a network traffic for which a rule is written in advance.

The instance of running `snort` software is called a sensor.
In order for `snort` to serve as an IPS the traffic must pass through the sensor.
`snort` then applies rules in order to block and/or log selected traffic.

`snort` IPS has been initially designed for sensor placing on a separate Linux machine, through which the traffic will pass
over a network bridge [https://openmaniak.com/inline_bridge.php](https://openmaniak.com/inline_bridge.php).

In order to place the sensor on an end-point host (the destination of the traffic,
or e.g. a load balancer) the recommended method is to send all traffic to be monitored
to `nfqueue` on this host and let `snort` apply rule based decisions to the packets present in `nfqueue`
[https://sourceforge.net/p/snort/mailman/message/35461638/](https://sourceforge.net/p/snort/mailman/message/35461638/)

In this setup the `nfqueue` (https://home.regit.org/netfilter-en/using-nfqueue-and-libnetfilter_queue/) `iptables` target is used to enable
the intrusion prevention capability of `snort`, and the `prometheus` (https://prometheus.io/) time-series database is used for monitoring of `snort` alerts.

`snort++` (or `snort3`), planned at least since 2005 (Snort Coockbook, p. 179) supports "multiple packet processing threads"
and several other parts of the code base have been rewritten compared to `snort2`. This setup uses `snort++`, though it is still *alpha*. After following the steps during this setup it is rather easy to switch to the more reliable `snort2`.

# Configuration overview

The private 192.168.17.0/24 network is associated with the Vagrant eth1 interfaces.
eth0 is used by Vagrant.  Vagrant reserves eth0 and this cannot be currently changed
(see https://github.com/mitchellh/vagrant/issues/2093).


                                                    -----------------
                                                    | monitor0      |
                                                    |               |
                                                    | prometheus    |
                                                    -----------------
                                                     |     |
                                                 eth1| eth0|
                                        192.168.17.20|     |
                 -----------------------------------------------------------------
                 |                                  Vagrant HOST                 |
                 -----------------------------------------------------------------
                  |     |                                         |     |         
              eth1| eth0|                                     eth1| eth0|         
     192.168.17.30|     |                            192.168.17.10|     |         
                 -----------------                               -----------------
                 | snort0        |                               | attacker0     |
                 |               |                               |               |
                 | snort         |                               -----------------
                 | node_exporter |
                 | apache        |
                 -----------------

Note that the target Apache host and the attacker machines reside on the same private network, while on the internet all addresses would be public.
This is in order to simplify routing, and it does no affect the `snort` configuration, i.e. a host configured like **snort0** would perform it's job if put on the internet.

`snort++` offers a multithreading mode. Note however that `snort`'s multithreading does not mean that a single `snort`
instance will internally load balance the traffic and process in multiple threads http://seclists.org/snort/2015/q2/90,
but instead multithreading means running multiple `snort` processes attached to different network interfaces, like one would do with `snort2` (is this statement correct?).
Obviously such multithreading configuration is not applicable to this setup.

# Sample Usage

Install VirtualBox https://www.virtualbox.org/ and Vagrant https://www.vagrantup.com/downloads.html

## Retrive the repository under the `jupyter` current working directory

In [226]:
%%bash
pwd

/scratch/ubuntu/opensourcedays


In [ ]:
%%bash
git clone https://github.com/marcindulak/vagrant-snort-nfqueue-tutorial-centos7

## Copy the `snort` setup files into the `jupyter` working directory

In [8]:
%%bash
cp -f vagrant-snort-tutorial-centos7/* .

Process is terminated.


## Bring up the VMs

 Note that the output appears only when the shell exits, be patient. It should take about 5-20 minutes on a modern hardware.

In [227]:
%%bash
vagrant up

Bringing machine 'attacker0' up with 'virtualbox' provider...
Bringing machine 'monitor0' up with 'virtualbox' provider...
Bringing machine 'snort0' up with 'virtualbox' provider...
==> attacker0: Importing base box 'bento/centos-7.3'...
==> attacker0: Matching MAC address for NAT networking...
==> attacker0: Checking if box 'bento/centos-7.3' is up to date...
==> attacker0: Setting the name of the VM: opensourcedays_attacker0_1488340562108_27147
==> attacker0: Clearing any previously set network interfaces...
==> attacker0: Preparing network interfaces based on configuration...
    attacker0: Adapter 1: nat
    attacker0: Adapter 2: hostonly
==> attacker0: Forwarding ports...
    attacker0: 22 (guest) => 2222 (host) (adapter 1)
==> attacker0: Running 'pre-boot' VM customizations...
==> attacker0: Booting VM...
==> attacker0: Waiting for machine to boot. This may take a few minutes...
    attacker0: SSH address: 127.0.0.1:2222
    attacker0: SSH username: vagrant
    attacker0: SSH aut

While waiting look at https://github.com/marcindulak/vagrant-snort-tutorial-centos7/blob/master/Vagrantfile. Notice that only a base system installation is performed on the VMs. The actual configuration of the machines is performed below.

## Configure `prometheus` on **monitor0**

In [228]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'sh /vagrant/configure_prometheus_on_monitor.sh'"

Detected operating system as centos/7.
Checking for curl...
Detected curl...
done.
Installing pygpgme to verify GPG signatures...
Package pygpgme-0.3-9.el7.x86_64 already installed and latest version
Installing yum-utils...
Package yum-utils-1.1.31-40.el7.noarch already installed and latest version
Generating yum cache for prometheus-rpm_release...

The repository is setup! You can now install packages.

 Package       Arch      Version                Repository                 Size
Installing:
 prometheus    x86_64    1.5.2-1.el7.centos     prometheus-rpm_release    9.6 M

Transaction Summary
Install  1 Package

Total download size: 9.6 M
Installed size: 48 M

Installed:
  prometheus.x86_64 0:1.5.2-1.el7.centos                                        



Created symlink from /etc/systemd/system/multi-user.target.wants/prometheus.service to /usr/lib/systemd/system/prometheus.service.


In [229]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'install -m 644 /vagrant/node_exporter.json /etc/prometheus'"

This configures the `prometheus` server on **monitor0**.

The `prometheus` server periodically pulls the metrics exposed by the nodes defined in `/etc/prometheus/node_exporter.json`.
Node exporter (https://github.com/prometheus/node_exporter) allows one to expose an arbitrary metric from a machine running the `node_exporter` service, using the functionality of Textfile Collector.

## Configure `snort` on **snort0**

In [230]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'install -m 644 /vagrant/copr-marcindulak-snort.repo /etc/yum.repos.d'"

In [231]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -y -d 1 -e 0 install snort snort-extra'"


 Package          Arch   Version                   Repository              Size
Installing:
 snort            x86_64 3.0.0-0.227.a4.el7.centos copr-marcindulak-snort 1.8 M
 snort-extra      x86_64 1.0.0-0.227.a4.el7.centos copr-marcindulak-snort 134 k
Installing for dependencies:
 daq              x86_64 2.2.1-1.el7.centos        copr-marcindulak-snort  86 k
 daq-modules      x86_64 2.2.1-1.el7.centos        copr-marcindulak-snort  37 k
 hwloc-libs       x86_64 1.11.2-1.el7              base                   1.5 M
 libdnet          x86_64 1.12-13.1.el7             base                    31 k
 libnetfilter_queue
                  x86_64 1.0.2-2.el7               epel                    23 k
 libtool-ltdl     x86_64 2.4.2-21.el7_2            base                    49 k
 luajit           x86_64 2.0.4-3.el7               epel                   343 k

Transaction Summary
Install  2 Packages (+7 Dependent packages)

Total download size: 3.9 M
Installed size: 11 M
Public key for libnetfil

Importing GPG key 0x37627D2F:
 Userid     : "marcindulak_snort (None) <marcindulak#snort@copr.fedorahosted.org>"
 Fingerprint: 987e e485 5793 87ee 3581 3bfb a919 6b3b 3762 7d2f
 From       : https://copr-be.cloud.fedoraproject.org/results/marcindulak/snort/pubkey.gpg
Importing GPG key 0x352C64E5:
 Userid     : "Fedora EPEL (7) <epel@fedoraproject.org>"
 Fingerprint: 91e9 7d7c 4a5e 96f1 7f3e 888f 6a2f aea2 352c 64e5
 Package    : epel-release-7-9.noarch (installed)
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7


In [232]:
%%bash
! vagrant ssh snort0 -c "sudo su - -c 'systemctl is-active snort@enp0s8'"

unknown


The exclamation mark above is used in order to assert that the command return with non-zero exit status.
We want a non-zero status because `snort` service should not be active yet. Note that `jupyter` cell does not "care" about the exit status, but if you run the command in the terminal or as part of as shell script the exit status does matter. 

## Disable affixing of unified2.log

The instance of `snort` is configured to log alerts and payloads which triggered the alerts in the `unified2` format (https://www.snort.org/faq/readme-unified2).

At every `snort` restart or when the maximum size of the log file is exceeded `snort` will create a `/var/log/snort/unified2.log.`date +%s` file, i.e. `/var/log/snort/unified2.log.` followed by the number of seconds elapsed since 1970-01-01 00:00:00 UTC (https://en.wikipedia.org/wiki/Unix_time).

This is inconvenient for the purpose of demonstrations and therefore adjust the `snort` configuration to always write to `/var/log/snort/unified2.log

In [233]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"echo unified2 = { limit = 128, units = \'M\', nostamp = true, mpls_event_types = true, vlan_event_types = true } >> /etc/snort/snort.lua\""""

## Workaround for http://seclists.org/snort/2017/q1/581 (the fix below seems insufficient)

In [234]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i 's|/usr/lib64/snort_extra|/usr/lib64/snort_extra/codecs|' /usr/lib/systemd/system/snort@.service\""

In [235]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl daemon-reload'"

## Start snort

In [236]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl start snort@enp0s8'"

In [237]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl is-active snort@enp0s8'"

active


"active" means `snort` is running now.

## Configure `node_exporter` for **snort0**

In [238]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'sh /vagrant/configure_node_exporter_on_snort.sh'"

Detected operating system as centos/7.
Checking for curl...
Detected curl...
done.
Installing pygpgme to verify GPG signatures...
Package pygpgme-0.3-9.el7.x86_64 already installed and latest version
Installing yum-utils...
Package yum-utils-1.1.31-40.el7.noarch already installed and latest version
Generating yum cache for prometheus-rpm_release...

The repository is setup! You can now install packages.

 Package         Arch     Version                Repository                Size
Installing:
 node_exporter   x86_64   0.13.0-1.el7.centos    prometheus-rpm_release   2.3 M

Transaction Summary
Install  1 Package

Total download size: 2.3 M
Installed size: 7.9 M

Installed:
  node_exporter.x86_64 0:0.13.0-1.el7.centos                                    



Created symlink from /etc/systemd/system/multi-user.target.wants/node_exporter.service to /usr/lib/systemd/system/node_exporter.service.


In [239]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl daemon-reload'"

In [240]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -d 1 -e 0 -y install python*-idstools python-dateutil'"


 Package                  Arch          Version               Repository   Size
Installing:
 python-dateutil          noarch        1.5-7.el7             base         85 k
 python2-idstools         noarch        0.5.4-1.el7           epel         76 k
 python34-idstools        noarch        0.5.4-1.el7           epel         78 k
Installing for dependencies:
 python34                 x86_64        3.4.5-3.el7           epel         50 k
 python34-libs            x86_64        3.4.5-3.el7           epel        6.7 M

Transaction Summary
Install  3 Packages (+2 Dependent packages)

Total download size: 7.0 M
Installed size: 28 M

Installed:
  python-dateutil.noarch 0:1.5-7.el7      python2-idstools.noarch 0:0.5.4-1.el7 
  python34-idstools.noarch 0:0.5.4-1.el7 

Dependency Installed:
  python34.x86_64 0:3.4.5-3.el7        python34-libs.x86_64 0:3.4.5-3.el7       



In [241]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'install -m 755 /vagrant/snort-alert-count.py /usr/local/bin'"

In [242]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'install -m 644 /vagrant/snort-alert-count.service /usr/lib/systemd/system'"

In [243]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl daemon-reload'"

In [244]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl enable snort-alert-count'"

Created symlink from /etc/systemd/system/multi-user.target.wants/snort-alert-count.service to /usr/lib/systemd/system/snort-alert-count.service.


In [245]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl start snort-alert-count'"

In [246]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl is-active snort-alert-count'"

active


In the context of `snort` this means: the `snort-alert-count.py` script continuously reads the `snort` alerts unified2 file generated by `snort` on **snort0** and writes the Textfile Collector file on **snort0** under `/var/lib/prometheus`.
The `node_exporter` service on **snort0** reads this file and exposes the metrics to the `prometheus` server.
  
Anyone willing to correct `snort-alert-count.py` so it writes the Textfile Collector file at regular time intervals, **independently** whether new alerts are arriving? Contributions are welcome.

# Snort usage

## Drop ICMP traffic

The classic example presented in most `snort` tutorials when explaining `snort` rules is to alert on `ping`. Our `snort` instance runs in IPS mode so we will be able to block 'pings'.

Here is a great introduction to `snort` rules "DevNet 1126 Detection Strategies with Snort" https://www.youtube.com/watch?v=-aeZ6OD8BPg

### Verify that `ping` works between `monitor0` and `snort0`

In [247]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'ping -c 1 -W 1 -I enp0s8 snort0'"

PING snort0 (192.168.17.30) from 192.168.17.20 enp0s8: 56(84) bytes of data.
64 bytes from snort0 (192.168.17.30): icmp_seq=1 ttl=64 time=6.95 ms

--- snort0 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 6.957/6.957/6.957/0.000 ms


### Copy the default `sample.rules` distributed by `snort` and enable them in `/etc/snort/snort_defaults.lua`

In [248]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cp -p /etc/snort/sample.rules /etc/snort/rules/snort.rules'"

In [249]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"echo RULE_PATH = conf_dir .. \'/rules\' >> /etc/snort/snort_defaults.lua\""  # http://seclists.org/snort/2017/q1/524

In [250]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"echo ips = { include = RULE_PATH .. \'/snort.rules\' } >> /etc/snort/snort_defaults.lua\""

### Enable dropping of incoming ICMP traffic on `snort0` by appending the appropriate rule to `/etc/snort/rules/sample.rules`

In [251]:
%%bash
echo 'drop icmp any any -> 192.168.17.30 any (msg:"icmp any any -> 192.168.17.30 any"; sid:3000001; rev:001;)' > 3000001.rules

In [252]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cat /vagrant/3000001.rules >> /etc/snort/rules/snort.rules'"

The `3000001` is the first Signature ID after the ET rules (https://rules.emergingthreats.net/), free for a custom use.

### Make `snort` to re-read the rules without restarting

In [253]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "kill -hup $(pidof snort)"'

In [254]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl is-active snort@enp0s8'"

active


### Verify that the ICMP traffic is now blocked by `snort0`

In [255]:
%%bash
! vagrant ssh monitor0 -c "sudo su - -c 'ping -c 1 -W 1 -I enp0s8 snort0'"

PING snort0 (192.168.17.30) from 192.168.17.20 enp0s8: 56(84) bytes of data.

--- snort0 ping statistics ---
1 packets transmitted, 0 received, 100% packet loss, time 0ms



If at this point the packet is **not** being dropped you may have another `snort0` or `monitor0` instance active somewhere from another `vagrant up`, or some other issue.

### Enable dropping of outgoing ICMP traffic on `snort0`

In [256]:
%%bash
echo 'drop icmp 192.168.17.30 any -> any any (msg:"icmp 192.168.17.30 any -> any any"; sid:3000002; rev:001;)' > 3000002.rules

In [257]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cat /vagrant/3000002.rules >> /etc/snort/rules/snort.rules'"

In [258]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "kill -hup $(pidof snort)"'

In [259]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'ping -c 1 -W 1 -I enp0s8 monitor0'"

PING monitor0 (192.168.17.20) from 192.168.17.30 enp0s8: 56(84) bytes of data.

--- monitor0 ping statistics ---
1 packets transmitted, 0 received, 100% packet loss, time 0ms



Note that the IDS functionality with `nfqueue` requires `snort` to run as `root`. In order to run `snort` in IDS mode change the user from `root` to `snort` and
remove the `-Q` option from `/usr/lib/systemd/system/snort@.service`.

## Display the alerts

Until now we should have two alerts showing the dropped incoming and outgoing ICMP packets. The details of the alert and the payload can be displayed using `u2spewfoo`

In [260]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'u2spewfoo /var/log/snort/unified2.log'"


(Event)
	sensor id: 0	event id: 1	event second: 1488341411	event microsecond: 158180
	sig id: 3000001	gen id: 1	revision: 1	 classification: 0
	priority: 0	ip source: 192.168.17.20	ip destination: 192.168.17.30
	src port: 0	dest port: 0	ip_proto: 1	impact_flag: 0	blocked: 0
	mpls label: 0	vland id: 0	policy id: 0

Packet
	sensor id: 0	event id: 1	event second: 1488341411
	packet second: 1488341411	packet microsecond: 158180
	linktype: 228	packet_length: 84
[    0] 45 00 00 54 9F D2 40 00 40 01 F7 53 C0 A8 11 14  E..T..@.@..S....
[   16] C0 A8 11 1E 08 00 BA D8 13 64 00 01 A3 49 B6 58  .........d...I.X
[   32] 00 00 00 00 0F 4D 02 00 00 00 00 00 10 11 12 13  .....M..........
[   48] 14 15 16 17 18 19 1A 1B 1C 1D 1E 1F 20 21 22 23  ............ !"#
[   64] 24 25 26 27 28 29 2A 2B 2C 2D 2E 2F 30 31 32 33  $%&'()*+,-./0123
[   80] 34 35 36 37                                      4567

(Event)
	sensor id: 0	event id: 2	event second: 1488341425	event microsecond: 175283
	sig id: 3000002	gen

An alternative method of listing the contents of the `unified2` log is performed using https://github.com/jasonish/py-idstools,
which converts `unified2` to `json` or `eve` (https://redmine.openinfosecfoundation.org/projects/suricata/wiki/_Logstash_Kibana_and_Suricata_JSON_output).

In [261]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'"

{"event": {"generator-id": 1, "vlan-id": 0, "source-ip": "192.168.17.20", "mpls-label": 0, "signature-id": 3000001, "signature-revision": 1, "impact": 0, "event-second": 1488341411, "event-microsecond": 158180, "sport-itype": 0, "destination-ip": "192.168.17.30", "priority": 0, "pad2": 0, "sensor-id": 0, "dport-icode": 0, "impact-flag": 0, "event-id": 1, "protocol": 1, "classification-id": 0, "blocked": 0}}
{"packet": {"event-id": 1, "sensor-id": 0, "data": "b'RQAAVJ/SQABAAfdTwKgRFMCoER4IALrYE2QAAaNJtlgAAAAAD00CAAAAAAAQERITFBUWFxgZGhscHR4fICEiIyQlJicoKSorLC0uLzAxMjM0NTY3'", "event-second": 1488341411, "length": 84, "packet-microsecond": 158180, "linktype": 228, "packet-second": 1488341411}}
{"event": {"generator-id": 1, "vlan-id": 0, "source-ip": "192.168.17.30", "mpls-label": 0, "signature-id": 3000002, "signature-revision": 1, "impact": 0, "event-second": 1488341425, "event-microsecond": 175283, "sport-itype": 0, "destination-ip": "192.168.17.20", "priority": 0, "pad2": 0, "sensor-id

Note that despite the fact that the packet has been dropped `unified2` does not provide this information as can be seen from the `u2spewfoo` output. As a consequence `idstools-u2json` assumes the packet has not been dropped. It seems worth inquiring the `snort-users` mailing list https://lists.sourceforge.net/lists/listinfo/snort-users whether the `unified2` missing *blocked* information is a bug, ar maybe a missing feature.

There is also tool `u2boat` for converting `unified2` into `pcap`, but does not seem to work as expected http://seclists.org/snort/2017/q1/11

In [262]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'u2boat /var/log/snort/unified2.log /tmp/u2boat.pcap'"

Defaulting to pcap output.


In [263]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -d 1 -e 0 -y install tcpdump'"


 Package          Arch            Version                   Repository     Size
Installing:
 tcpdump          x86_64          14:4.5.1-3.el7            base          387 k

Transaction Summary
Install  1 Package

Total download size: 387 k
Installed size: 931 k

Installed:
  tcpdump.x86_64 14:4.5.1-3.el7                                                 



In [264]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'tcpdump -XX -r /tmp/u2boat.pcap'" | grep 'ethertype Unknown'

04:10:11.158180 40:00:40:01:f7:53 (oui Unknown) > 45:00:00:54:9f:d2 (oui Unknown), ethertype Unknown (0xc0a8), length 84: 
04:10:25.175283 40:00:40:01:6e:7d (oui Unknown) > 45:00:00:54:28:a9 (oui Unknown), ethertype Unknown (0xc0a8), length 84: 


reading from file /tmp/u2boat.pcap, link-type EN10MB (Ethernet)


See "Introduction to Packet Analysis" https://www.youtube.com/watch?v=visrNiKIP3E&index=17 for a refresher about network protocols and
tools like `tcpdump` or `wireshark`.

The `eve` format is used when pushing `snort` logs into ELK (https://www.elastic.co/products), or for converting `unified2` into `pcap`

In [265]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "python2 $(which idstools-u2eve) /var/log/snort/unified2.log > /tmp/u2eve.log"'

In [266]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i 's/-0600/+0000/' /usr/lib/python2.7/site-packages/idstools/scripts/eve2pcap.py\""  # https://github.com/jasonish/py-idstools/issues/37

In [267]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "python2 $(which idstools-eve2pcap) /tmp/u2eve.log --dlt RAW -o /tmp/u2eve2pcap.pcap"'

In [268]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'tcpdump -XX -r /tmp/u2eve2pcap.pcap'" | grep 'ICMP'

04:10:11.158180 IP monitor0 > snort0: ICMP echo request, id 4964, seq 1, length 64
04:10:25.175283 IP snort0 > monitor0: ICMP echo request, id 5966, seq 1, length 64


reading from file /tmp/u2eve2pcap.pcap, link-type RAW (Raw IP)


## Periodic test alert

A periodic test alert can be setup (http://ossectools.blogspot.dk/2011/04/network-intrusion-detection-systems.html) the purpose of which
will be to probe whether `snort` or another resource on `snort0` is able to keep up with the traffic. If the rate of the test alert
falls below a threshold we will know that some alerts were missed by `snort`. An HTTP request on port 80 will be used as a test alert.

### In order for the rule to triggered a web server must be listening on port 80 on `snort0`

In [269]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -d 1 -e 0 -y install httpd'"


 Package            Arch          Version                     Repository   Size
Installing:
 httpd              x86_64        2.4.6-45.el7.centos         base        2.7 M
Installing for dependencies:
 apr                x86_64        1.4.8-3.el7                 base        103 k
 apr-util           x86_64        1.5.2-6.el7                 base         92 k
 httpd-tools        x86_64        2.4.6-45.el7.centos         base         84 k
 mailcap            noarch        2.1.41-2.el7                base         31 k

Transaction Summary
Install  1 Package (+4 Dependent packages)

Total download size: 3.0 M
Installed size: 10 M

Installed:
  httpd.x86_64 0:2.4.6-45.el7.centos                                            

Dependency Installed:
  apr.x86_64 0:1.4.8-3.el7                     apr-util.x86_64 0:1.5.2-6.el7    
  httpd-tools.x86_64 0:2.4.6-45.el7.centos     mailcap.noarch 0:2.1.41-2.el7    



In [270]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl enable httpd'"

Created symlink from /etc/systemd/system/multi-user.target.wants/httpd.service to /usr/lib/systemd/system/httpd.service.


### The rule must be added to detect the test alert

The proper rule would like like *alert tcp 192.168.17.20 any -> any 80 (msg:"snort-test-alert"; flow:to_server,established; http_uri; content:"/snort-test-alert"; classtype:not-suspicious; sid:3000003; rev:001;)* but due to http://seclists.org/snort/2017/q1/581 it is modified

In [271]:
%%bash
echo 'alert tcp 192.168.17.20 any -> any 80 (msg:"snort-test-alert"; content:"/snort-test-alert"; classtype:not-suspicious; sid:3000003; rev:001;)' > 3000003.rules

In [272]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cat /vagrant/3000003.rules >> /etc/snort/rules/snort.rules'"

In [273]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "kill -hup $(pidof snort)"'

The HTTP request is sent using `curl -s -m 1 http://snort0/snort-test-alert` from `monitor0`, let's save it's `pcap` on `snort0`. The `pcap` will be later used to
test how much traffic `snort0` can handle by replaying the traffic with `tcpreplay`

In order to capture the HTTP traffic Apache must be listening

In [274]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl start httpd'"

In [275]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'yum -d 1 -e 0 -y install at'"


 Package       Arch              Version                  Repository       Size
Installing:
 at            x86_64            3.1.13-22.el7            base             51 k

Transaction Summary
Install  1 Package

Total download size: 51 k
Installed size: 95 k

Installed:
  at.x86_64 0:3.1.13-22.el7                                                     



In [276]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl start atd'"

In [277]:
%%bash
vagrant ssh monitor0 -c "echo 'curl -s -m 1 http://snort0/snort-test-alert' | at now + 1 minute"

job 1 at Wed Mar  1 04:13:00 2017


In [278]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'timeout 70 tcpdump -i enp0s8 port 80 -w /vagrant/snort-test-alert.pcap'"

tcpdump: listening on enp0s8, link-type EN10MB (Ethernet), capture size 65535 bytes
10 packets captured
10 packets received by filter
0 packets dropped by kernel


We expect 10 packets to be captured. Let's stop Apache for the purpose of demonstrations in the next section.

In [279]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl stop httpd'"

### After verifying the test alert is being detected by `snort` enable the `snort-test-alert` service

In [280]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'install -m 644 /vagrant/snort-test-alert.service /usr/lib/systemd/system'"

In [281]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl daemon-reload'"

In [282]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl enable snort-test-alert'"

Created symlink from /etc/systemd/system/multi-user.target.wants/snort-test-alert.service to /usr/lib/systemd/system/snort-test-alert.service.


In [283]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl start snort-test-alert'"

In [284]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl is-active snort-test-alert'"

active


The test alert should keep appearing in the `tcpdump` output on `snort0`

In [285]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'timeout 10 tcpdump -i enp0s8 port 80'" | grep snort0.http

04:13:47.685548 IP monitor0.42148 > snort0.http: Flags [S], seq 330953197, win 29200, options [mss 1460,sackOK,TS val 657430 ecr 0,nop,wscale 6], length 0
04:13:47.685964 IP snort0.http > monitor0.42148: Flags [R.], seq 0, ack 330953198, win 0, length 0


tcpdump: verbose output suppressed, use -v or -vv for full protocol decode
listening on enp0s8, link-type EN10MB (Ethernet), capture size 65535 bytes
2 packets captured
2 packets received by filter
0 packets dropped by kernel


but it is not yet detectable by `snort` (because `httpd` is not running, only `tcp` traffic is exchanged)

In [287]:
%%bash
! vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep '"signature-id": 3000003'

{"event": {"event-id": 3, "sport-itype": 42132, "blocked": 0, "event-microsecond": 562113, "pad2": 0, "event-second": 1488341580, "vlan-id": 0, "impact-flag": 0, "classification-id": 1, "mpls-label": 0, "destination-ip": "192.168.17.30", "sensor-id": 0, "impact": 0, "protocol": 6, "source-ip": "192.168.17.20", "signature-id": 3000003, "dport-icode": 80, "signature-revision": 1, "generator-id": 1, "priority": 3}}


### After Apache is started the test alerts will be detected by `snort`

In [288]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl start httpd&& sleep 10'"

In [289]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep '"signature-id": 3000003'

{"event": {"dport-icode": 80, "signature-revision": 1, "impact": 0, "blocked": 0, "event-second": 1488341580, "event-id": 3, "mpls-label": 0, "vlan-id": 0, "classification-id": 1, "sensor-id": 0, "signature-id": 3000003, "generator-id": 1, "destination-ip": "192.168.17.30", "source-ip": "192.168.17.20", "impact-flag": 0, "protocol": 6, "priority": 3, "pad2": 0, "sport-itype": 42132, "event-microsecond": 562113}}
{"event": {"dport-icode": 80, "signature-revision": 1, "impact": 0, "blocked": 0, "event-second": 1488341767, "event-id": 4, "mpls-label": 0, "vlan-id": 0, "classification-id": 1, "sensor-id": 0, "signature-id": 3000003, "generator-id": 1, "destination-ip": "192.168.17.30", "source-ip": "192.168.17.20", "impact-flag": 0, "protocol": 6, "priority": 3, "pad2": 0, "sport-itype": 42212, "event-microsecond": 821582}}
{"event": {"dport-icode": 80, "signature-revision": 1, "impact": 0, "blocked": 0, "event-second": 1488341777, "event-id": 5, "mpls-label": 0, "vlan-id": 0, "classificat

## Snort alerts in Prometheus

Prometheus is not best suited for storing and analyzing `snort` alerts but the simplicity of `prometheus` configuration fits well into this tutorial.
For an introduction to `prometheus` see "An introduction to monitoring and alerting with time-series at scale, with Prometheus" https://www.youtube.com/watch?v=gNmWzkGViAY

The `snort-alert-count.py` script on `snort0` continuously reads the output produced by `idstools-u2json /var/log/snort/unified2.log` and
periodically write the `/var/lib/prometheus/snort.prom` file with `snort` alerts counter.

In [290]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'grep snort_alert_count /var/lib/prometheus/snort.prom'"

# HELP snort_alert_count Snort alert count.
# TYPE snort_alert_count counter
snort_alert_count{generator_id="1",signature_id="3000001",blocked="0",source_ip="192.168.17.20",dport_icode="0"}  1
snort_alert_count{generator_id="1",signature_id="3000002",blocked="0",source_ip="192.168.17.30",dport_icode="0"}  1
snort_alert_count{generator_id="1",signature_id="3000003",blocked="0",source_ip="192.168.17.20",dport_icode="80"}  6


After several minutes `snort` alerts will start appearing in `prometheus`

In [295]:
%%bash
sleep 300

Process is interrupted.


In [291]:
%%bash
vagrant ssh monitor0 -c "curl -sgG --data-urlencode query='rate(snort_alert_count[5m]) * 60' localhost:9090/api/v1/query"

{"status":"success","data":{"resultType":"vector","result":[{"metric":{"blocked":"0","dport_icode":"0","generator_id":"1","instance":"snort0:9100","job":"node","signature_id":"3000001","source_ip":"192.168.17.20"},"value":[1488341855.515,"0"]},{"metric":{"blocked":"0","dport_icode":"0","generator_id":"1","instance":"snort0:9100","job":"node","signature_id":"3000002","source_ip":"192.168.17.30"},"value":[1488341855.515,"0"]},{"metric":{"blocked":"0","dport_icode":"80","generator_id":"1","instance":"snort0:9100","job":"node","signature_id":"3000003","source_ip":"192.168.17.20"},"value":[1488341855.515,"1.4919931276471723"]}]}}

The `prometheus` query 'rate(snort_alert_count[5m]) * 60' returns the average rate of increase of `snort` alerts (per alert) per minute, during the last 5 minutes
(https://prometheus.io/docs/querying/functions/#rate()). The `snort-test-alert` on `monitor0` performs `curl` against `snort0` every 10 seconds
so such defined rate of the test alert should be equal to 6 per minute.

Prometheus has it's own alerting system, but one can simply query the HTTP API of `prometheus` and include such checks into traditional
monitoring systems like `nagios`

In [292]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'yum -d 1 -e 0 -y install git jq'"


 Package                Arch         Version                   Repository  Size
Installing:
 git                    x86_64       1.8.3.1-6.el7_2.1         base       4.4 M
 jq                     x86_64       1.5-1.el7                 epel       153 k
Installing for dependencies:
 libgnome-keyring       x86_64       3.8.0-3.el7               base       109 k
 oniguruma              x86_64       5.9.5-3.el7               epel       129 k
 perl-Error             noarch       1:0.17020-2.el7           base        32 k
 perl-Git               noarch       1.8.3.1-6.el7_2.1         base        53 k
 perl-TermReadKey       x86_64       2.30-20.el7               base        31 k
 rsync                  x86_64       3.0.9-17.el7              base       360 k

Transaction Summary
Install  2 Packages (+6 Dependent packages)

Total download size: 5.2 M
Installed size: 24 M
Public key for jq-1.5-1.el7.x86_64.rpm is not installed

Installed:
  git.x86_64 0:1.8.3.1-6.el7_2.1              jq.x86_64 

Importing GPG key 0x352C64E5:
 Userid     : "Fedora EPEL (7) <epel@fedoraproject.org>"
 Fingerprint: 91e9 7d7c 4a5e 96f1 7f3e 888f 6a2f aea2 352c 64e5
 Package    : epel-release-7-9.noarch (installed)
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7


In [293]:
%%bash
vagrant ssh monitor0 -c "git clone https://github.com/prometheus/nagios_plugins"

Cloning into 'nagios_plugins'...


In [299]:
%%bash
vagrant ssh monitor0 -c "bash ./nagios_plugins/check_prometheus_metric.sh -H localhost:9090 -q \"rate(snort_alert_count{signature_id='3000003'}[5m]) * 60\" -w 5 -c 4 -n snort-test-alert -m lt -t vector -i"

OK - snort-test-alert is 5: { blocked: 0, dport_icode: 80, generator_id: 1, instance: snort0:9100, job: node, signature_id: 3000003, source_ip: 192.168.17.20 }


This query will return the "WARNING" when the rate of the test alert drops below 5 and "CRITICAL" when it drops below 4.

Over the years several domain-specific languages (DSL) have been proposed for making the alerts produced by and IDS/IPS more manageable.
One can use `prometheus` as a poor man's replacement of such a system. For example, it is interesting to know whether there are any
persistent attackers on the network, who accesses the system over an extended period of time using various or the same types of attacks.

The query below will identify the source IP numbers which triggered an alert during the last 5 minutes, and were also triggering some alerts
(not necessarily the same) during the preceding hour:

((sum without (signature-id,generator-id,dport-icode)(rate(snort_alert_count{alert_id!="1:3000000"}[5m]))) > 0 and (sum without (signature-id,generator-id,dport-icode)(rate(snort_alert_count{alert_id!="1:3000000"}[1h] offset 5m))) > 0) * 60

Another query will provide a complementary information about the types of alerts, which tiggered during the last 5 minutes, and were
also triggered during the preceding hour, independently of the source IP number:

((sum without (source-ip,dport-icode)(rate(snort_alert_count{alert_id!="1:3000000"}[5m]))) > 0 and (sum without (source-ip,dport-icode)(rate(snort_alert_count{alert_id!="1:3000000"}[1h] offset 5m))) > 0) * 60

See "PromCon 2016: Alerting in the Prometheus Universe - Fabian Reinartz" https://www.youtube.com/watch?v=yrK6z3fpu1E for more details about alerting in `prometheus`.

## Manage rules with Pulledpork

Pulledpork [https://github.com/shirkdog/pulledpork](https://github.com/shirkdog/pulledpork) writes the selected rules into `/etc/snort/rules/snort.rules`.

Unfortunately certain rules options have been modified/removed from `snort++` (see https://github.com/snortadmin/snort3/blob/master/doc/differences.txt),
which means that most `snort2` are unusable. Just skip this section. The instructions in this section should work with `snort2` however.

`snortrules-snapshot-*.tar.gz` are available for free after registration at https://www.snort.org/downloads, and are divided into `categories`.
On the oter hand the `community-rules.tar.gz` available without registration are not divided into `categories`, but still use `categories` labels in the `msg` field.

Pulledpork works in a slightly complicated way when `categories` are involved (I hope I got it right, if not corrections are welcome).
The behavior is dependent on the setting of the `order` variable in `/etc/pulledpork/pulledpork.conf`, and for `order=disable,enable,drop`:

- all rules that are uncommented in the files distributed within a rules tarball are written into `/etc/snort/rules/snort.rules`

- a category listed in `/etc/pulledpork/enablesid.conf` will enable all rules from the category (also the commented)

- a category listed in `/etc/pulledpork/disablesid.conf` will disable all rules from the category (also the commented)

- a category listed in `/etc/pulledpork/dropsid.conf` will convert uncommented rules from the category to drop, and also the commented rules if the category is in `/etc/pulledpork/enablesid.conf`

- `/etc/pulledpork/modifysid.conf` can be used to modify the rules, like replacing ports or other sequences of characters in the rule

The list of categories and how to create the list manually is here
[https://github.com/shirkdog/pulledpork/blob/master/doc/README.CATEGORIES](https://github.com/shirkdog/pulledpork/blob/master/doc/README.CATEGORIES)

Let's enable `community-rules.tar.gz` using `pulledpork`, enabling few categories, disabled (commented) by default

In [300]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -d 1 -e 0 -y install pulledpork'"


 Package                     Arch       Version                  Repository
                                                                           Size
Installing:
 pulledpork                  noarch     0.7.2-2.el7              epel      44 k
Installing for dependencies:
 perl-Archive-Tar            noarch     1.92-2.el7               base      73 k
 perl-Business-ISBN          noarch     2.06-2.el7               base      25 k
 perl-Business-ISBN-Data     noarch     20120719.001-2.el7       base      24 k
 perl-Compress-Raw-Bzip2     x86_64     2.061-3.el7              base      32 k
 perl-Compress-Raw-Zlib      x86_64     1:2.061-4.el7            base      57 k
 perl-Crypt-SSLeay           x86_64     0.64-5.el7               base      58 k
 perl-Data-Dumper            x86_64     2.145-3.el7              base      47 k
 perl-Digest                 noarch     1.17-245.el7             base      23 k
 perl-Digest-MD5             x86_64     2.52-3.el7               base      30 k
 p

In [301]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i '/IPBLACKLIST/d' /etc/pulledpork/pulledpork.conf\""  # we don't use IP blacklisting feature

In [302]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'echo pcre:INDICATOR-SCAN >> /etc/pulledpork/enablesid.conf'"

In [303]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'echo pcre:SERVER-APACHE >> /etc/pulledpork/enablesid.conf'"

In [304]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'echo pcre:SERVER-WEBAPP >> /etc/pulledpork/enablesid.conf'"

Fetch and install the selected rules, under `/tmp/snort.rules` - on our setup we don't want to overwrite `/etc/snort/rules/snort.rules`!

In [305]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'pulledpork -PE -c /etc/pulledpork/pulledpork.conf -o /tmp/snort.rules'"

 
    https://github.com/shirkdog/pulledpork
      _____ ____
     `----,\    )
      `--==\\  /    PulledPork v0.7.2 - E.Coli in your water bottle!
       `--==\\/
     .-~~~~-.Y|\\_  Copyright (C) 2009-2016 JJ Cummings
  @_/        /  66\_  cummingsj@gmail.com
    |    \   \   _(")
     \   /-| ||'--'  Rules give me wings!
      \_\  \_\\
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Checking latest MD5 for community-rules.tar.gz....
Rules tarball download of community-rules.tar.gz....
	They Match
	Done!
Checking latest MD5 for opensource.tar.gz....
Rules tarball download of opensource.tar.gz....
	They Match
	Done!
Prepping rules from community-rules.tar.gz for work....
	Done!
Prepping rules from opensource.tar.gz for work....
	Done!
Reading rules...
Modifying Sids....
	Done!
Processing /etc/pulledpork/enablesid.conf....
	Modified 831 rules
	Skipped 831 rules (already disabled)
	Done
Processing /etc/pulledpork/dropsid.conf....
	Modified 0 rules
	Skipped 0 rules (al

## Examples of actions from the attacker machine

This subject is virtually unlimited, but a representative set of actions detectable by `snort` is available at https://www.aldeid.com/wiki/Suricata-vs-snort

### Portscans

The "SYN" scan consists of finding out whether a TCP port is open by sending "SYN".
The port is considered open when "SYN-ACK" or "SYN" is received https://nmap.org/book/man-port-scanning-techniques.html
The rule detecting "SYN" scan is in `community-rules.tar.gz`, but since many `snort2` rules are incompatible with `snort++` let's take
the rules explicitly from `community-rules.tar.gz`

In [306]:
%%bash
echo 'alert tcp $EXTERNAL_NET any -> $HOME_NET any (msg:"INDICATOR-SCAN synscan portscan"; flow:stateless; flags:SF; id:39426; metadata:ruleset community; classtype:attempted-recon; sid:630; rev:10;)' > 630.rules

In [307]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cat /vagrant/630.rules >> /etc/snort/rules/snort.rules'"

In [308]:
%%bash
vagrant ssh snort0 -c 'sudo su - -c "kill -hup $(pidof snort)"'

Perform the scan from `attacker0`

In [309]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "yum -d 1 -e 0 -y install nmap"'


 Package            Arch            Version                 Repository     Size
Installing:
 nmap               x86_64          2:6.40-7.el7            base          4.0 M
Installing for dependencies:
 nmap-ncat          x86_64          2:6.40-7.el7            base          201 k

Transaction Summary
Install  1 Package (+1 Dependent package)

Total download size: 4.2 M
Installed size: 17 M

Installed:
  nmap.x86_64 2:6.40-7.el7                                                      

Dependency Installed:
  nmap-ncat.x86_64 2:6.40-7.el7                                                 



In [310]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "nmap -sS snort0"'


Starting Nmap 6.40 ( http://nmap.org ) at 2017-03-01 04:22 UTC
Nmap scan report for snort0 (192.168.17.30)
Host is up (0.0012s latency).
Not shown: 996 closed ports
PORT     STATE SERVICE
22/tcp   open  ssh
80/tcp   open  http
111/tcp  open  rpcbind
9100/tcp open  jetdirect
MAC Address: 08:00:27:00:17:30 (Cadmus Computer Systems)

Nmap done: 1 IP address (1 host up) scanned in 0.10 seconds


The "SYN" scan seems undetected, even after increasing the sensitivity to scans in `/etc/snort/snort.lua` with `port_scan.sense_level = 'high'`. An issue worth asking about at `snort-users`.

In [311]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep -v packet | grep '"signature-id": 663'

### Tcpreplay

Here `tcpreplay` is used from the `attacker0` machine, this is actually not an attack.
The reason for using `attacker0` is the amount of RAM needed for caching of packets in `tcpeplay`, but also to demonstrate the purpose of `tcprewrite`.

We will be pretending to be `monitor0` from `attacker0`. In order to demonstrate this let's disable the `snort-test-alert` on `monitor0`

In [312]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl stop snort-test-alert'"

and get the total number of tiggered test alerts until now. All of them were triggered by the traffic from `monitor0` to `snort0`

In [313]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep -v packet | grep '"signature-id": 3000003' | grep '192.168.17.30' | wc -l

41


Let's replay the test alert 100000 times aiming at 200 Mbps (which may or may not be reached, depending on the speed of you RAM) from the original `pcap`

In [314]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "yum -d 1 -e 0 -y install tcpreplay"'


 Package            Arch            Version                 Repository     Size
Installing:
 tcpreplay          x86_64          4.1.2-1.el7             epel          299 k
Installing for dependencies:
 tcpdump            x86_64          14:4.5.1-3.el7          base          387 k

Transaction Summary
Install  1 Package (+1 Dependent package)

Total download size: 687 k
Installed size: 2.2 M
Public key for tcpreplay-4.1.2-1.el7.x86_64.rpm is not installed

Installed:
  tcpreplay.x86_64 0:4.1.2-1.el7                                                

Dependency Installed:
  tcpdump.x86_64 14:4.5.1-3.el7                                                 



Importing GPG key 0x352C64E5:
 Userid     : "Fedora EPEL (7) <epel@fedoraproject.org>"
 Fingerprint: 91e9 7d7c 4a5e 96f1 7f3e 888f 6a2f aea2 352c 64e5
 Package    : epel-release-7-9.noarch (installed)
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7


In [315]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "tcpreplay -K -l 100000 -M 200 --intf1 enp0s8 /vagrant/snort-test-alert.pcap"'

Actual: 1000000 packets (113600000 bytes) sent in 6.00 seconds.
Rated: 18830200.0 Bps, 150.64 Mbps, 165759.21 pps
Flows: 2 flows, 0.33 fps, 1000000 flow packets, 0 non-flow
Statistics for network device: enp0s8
	Successful packets:        1000000
	Failed packets:            0
	Truncated packets:         0
	Retried packets (ENOBUFS): 0
	Retried packets (EAGAIN):  0


File Cache is enabled


Note that the `attacker0` IP is not detected by `snort0`, but instead the count for alerts from `monitor0` increases significantly

In [316]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep -v packet | grep '"signature-id": 3000003' | grep '192.168.17.10' | wc -l

0


In [317]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep -v packet | grep '"signature-id": 3000003' | grep '192.168.17.30' | wc -l

4092


After this test make sure the `prometheus` level of the test snort alert is restored

In [318]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl start snort-test-alert'"

In [319]:
%%bash
vagrant ssh monitor0 -c "bash ./nagios_plugins/check_prometheus_metric.sh -H localhost:9090 -q \"rate(snort_alert_count{source_ip='192.168.17.20',signature_id='3000003'}[5m]) * 60\" -w 5 -c 4 -n snort-test-alert -m lt -t vector -i"

OK - snort-test-alert is 6: { blocked: 0, dport_icode: 80, generator_id: 1, instance: snort0:9100, job: node, signature_id: 3000003, source_ip: 192.168.17.20 }


Let's now `tcprewrite` the source in the `snort-test-alert.pcap` to honestly report that we are an `attacker0`. `tcprewrite` of the destination would be necessary if `pcap` has been captured on a different machine than `snort0`

In [320]:
%%bash
vagrant ssh attacker0 -c 'tcprewrite --infile=/vagrant/snort-test-alert.pcap --outfile=/tmp/1.pcap --dstipmap=0.0.0.0/0:192.168.17.30 --enet-dmac=08:00:27:00:17:30 --portmap=80:80'

In [321]:
%%bash
vagrant ssh attacker0 -c 'tcprewrite --infile=/tmp/1.pcap --outfile=/tmp/2.pcap --srcipmap=0.0.0.0/0:192.168.17.10 --enet-smac=08:00:27:00:17:10'

Replay it, this time with 20 times more packets, this will take about 5 minutes

In [322]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "tcpreplay -K -l 2000000 -M 200 --intf1 enp0s8 /tmp/2.pcap"'

Actual: 20000000 packets (2272000000 bytes) sent in 150.07 seconds.
Rated: 15075300.0 Bps, 120.60 Mbps, 132705.94 pps
Flows: 2 flows, 0.01 fps, 20000000 flow packets, 0 non-flow
Statistics for network device: enp0s8
	Successful packets:        20000000
	Failed packets:            0
	Truncated packets:         0
	Retried packets (ENOBUFS): 0
	Retried packets (EAGAIN):  0


File Cache is enabled


The CPU/Memory usage by `snort` on `snort0` increases significantly (100% CPU) during the replay at a modest 200 Mbps. You can always login to any of the machines, e.g. **vagrant ssh snort0**. To gain root use **sudo su -**

In [323]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'idstools-u2json /var/log/snort/unified2.log'" | grep -v packet | grep '"signature-id": 3000003' | grep '192.168.17.30' | wc -l

4104


Let's quickly check count of test alerts, the rate should be "CRITICAL"

In [324]:
%%bash
! vagrant ssh monitor0 -c "bash ./nagios_plugins/check_prometheus_metric.sh -H localhost:9090 -q \"rate(snort_alert_count{source_ip='192.168.17.20',signature_id='3000003'}[5m]) * 60\" -w 5 -c 4 -n snort-test-alert -m lt -t vector -i"

CRITICAL - snort-test-alert is 3: { blocked: 0, dport_icode: 80, generator_id: 1, instance: snort0:9100, job: node, signature_id: 3000003, source_ip: 192.168.17.20 }


Sometimes people blame `nfqueue` for dropping packets. The sixth column of `/proc/net/netfilter/nfnetlink_queue` reports the number of packets dropped by `nfqueue` - it is zero

In [325]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'cat /proc/net/netfilter/nfnetlink_queue'"

    0   5123     0 2 65531     0 17630422 25621151  1


It is an interesting subject for a discussion at `snort-users` what are the factors limiting the performance of this setup.

### Pytbull

http://pytbull.sourceforge.net/ is a `python` module which automates scans against `snort`/`suricata` IDS/IPS.

In [326]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "yum -d 1 -e 0 -y install mercurial"'


 Package            Arch            Version                 Repository     Size
Installing:
 mercurial          x86_64          2.6.2-6.el7_2           base          2.6 M

Transaction Summary
Install  1 Package

Total download size: 2.6 M
Installed size: 12 M

Installed:
  mercurial.x86_64 0:2.6.2-6.el7_2                                              



In [327]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "yum -d 1 -e 0 -y install python-feedparser python-cherrypy nikto hping3 tcpreplay httpd-tools python-paramiko ncrack scapy"'

Package tcpreplay-4.1.2-1.el7.x86_64 already installed and latest version
Package python-paramiko is obsoleted by python2-paramiko, trying to install python2-paramiko-1.16.1-2.el7.noarch instead

 Package                Arch        Version                     Repository
                                                                           Size
Installing:
 hping3                 x86_64      0.0.20051105-24.el7         epel       96 k
 httpd-tools            x86_64      2.4.6-45.el7.centos         base       84 k
 ncrack                 x86_64      0.4-0.11.ALPHA.el7          epel      554 k
 nikto                  noarch      1:2.1.5-10.el7              epel      263 k
 python-cherrypy        noarch      3.2.2-4.el7                 base      422 k
 python-feedparser      noarch      5.1.3-3.el7                 epel      107 k
 python2-paramiko       noarch      1.16.1-2.el7                epel      258 k
 scapy                  noarch      2.2.0-2.el7                 epel      489

In [328]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "hg clone http://pytbull.hg.sourceforge.net:8000/hgroot/pytbull/pytbull"'

destination directory: pytbull
requesting all changes
adding changesets
adding manifests
adding file changes
added 17 changesets with 199 changes to 148 files
updating to branch default
139 files updated, 0 files merged, 0 files removed, 0 files unresolved


In [329]:
%%bash
vagrant ssh attacker0 -c 'sudo su - -c "cp -f /vagrant/pytbull_config.cfg ~/pytbull/conf/config.cfg"'

Let's stop the test alert not to confuse `pytbull`

In [331]:
%%bash
vagrant ssh monitor0 -c "sudo su - -c 'systemctl stop snort-test-alert'"

`pytbull` is expecting to have access to `snort0:/var/log/snort/alert_fast.txt` in the `-A fast` format

In [332]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl stop snort@enp0s8'"

In [333]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i 's/-A unified2/-A fast/' /usr/lib/systemd/system/snort@.service\""

In [334]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl daemon-reload'"

In [335]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i 's/^unified2 =/--unified2 =/' /etc/snort/snort.lua\""

In [336]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"echo alert_fast = { file = true, limit = 128, units = \'M\' } >> /etc/snort/snort.lua\""""

In [337]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'rm -f /var/log/snort/alert_fast.txt'"

Remove our ping drops, as they will fire on some `pytbull` tests

In [338]:
%%bash
vagrant ssh snort0 -c "sudo su - -c \"sed -i '/drop icmp/d' /etc/snort/rules/snort.rules\""

In [339]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl start snort@enp0s8'"

In [340]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'systemctl is-active snort@enp0s8'"

active


Moreover it wants an `ftp` server on the `snort0` sensor to access `/var/log/snort/alert_fast.txt`. This is not something one wants on a production sensor

In [341]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'yum -d 1 -e 0 -y install vsftpd&& systemctl start vsftpd'"


 Package          Arch             Version                 Repository      Size
Installing:
 vsftpd           x86_64           3.0.2-21.el7            base           169 k

Transaction Summary
Install  1 Package

Total download size: 169 k
Installed size: 348 k

Installed:
  vsftpd.x86_64 0:3.0.2-21.el7                                                  



In [342]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'getent passwd pilou > /dev/null || useradd pilou'"

In [343]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'echo oopsoops | passwd --stdin pilou'"

Changing password for user pilou.
passwd: all authentication tokens updated successfully.


In [344]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'chmod go+rx /var/log/snort'"

In [345]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'chmod go+r /var/log/snort/alert_fast.txt'"

Test whether `snort` is still able to detect the test alert

In [348]:
%%bash
vagrant ssh monitor0 -c "curl -s -m 1 http://snort0/snort-test-alert"

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /snort-test-alert was not found on this server.</p>
</body></html>


In [347]:
%%bash
vagrant ssh snort0 -c "sudo su - -c 'grep 3000003 /var/log/snort/alert_fast.txt'"

03/01-04:34:51.219343 [**] [1:3000003:1] "snort-test-alert" [**] [Classification: Not Suspicious Traffic] [Priority: 3] {TCP} 192.168.17.20:42702 -> 192.168.17.30:80


Run `pytbull`, it should takes less than 5 minutes

In [217]:
%%bash
vagrant ssh attacker0 -c "sudo su - -c \"cd /root/pytbull; echo -e '1\ny\n' | timeout 720 ./pytbull --debug --offline --mode=gateway -t snort0\""

Process is interrupted.


`pytbull` starts a web server and exposes a summary of the scan as html, but in this setup it finds no alerts are triggered. Something is not right here.

# When done, destroy the test machines

In [ ]:
%%bash
vagrant destroy -f

# License

BSD 2-clause